<a href="https://colab.research.google.com/github/omar907/Skin-Cancer-Classfication/blob/master/Skin_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skin Cancer Detection

In [0]:
#Install Tensorflow 2 to use this nootbook
!pip install tensorflow==2.0.0-alpha0 

# MOUNT DRIVE to get the folder of the data
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content
!git clone https://github.com/vogon101/skincancer.git

/content
Cloning into 'skincancer'...
remote: Enumerating objects: 553, done.
remote: Total 553 (delta 0), reused 0 (delta 0), pack-reused 553
Receiving objects: 100% (553/553), 294.58 MiB | 42.72 MiB/s, done.
Resolving deltas: 100% (309/309), done.
Checking out files: 100% (103/103), done.


In [0]:
import os

train_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/train'

train_Benign_dir = os.path.join(train_dir,'Benign')
train_Malignant_dir = os.path.join(train_dir,'Malignant')


validation_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/validation'

validation_Benign_dir = os.path.join(validation_dir,'Benign')
validation_Malignant_dir = os.path.join(validation_dir,'Malignant')


%cd /content/drive/My Drive/Datasets/Skin_Detection

/content/drive/My Drive/Datasets/Skin_Detection


#Model from Scratch (3 Conv. Layers)

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import sys

Using TensorFlow backend.


In [0]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),     # Dropout layer for Overfitting
    keras.layers.Dense(1, activation = 'sigmoid')    
])
model.summary()

W0717 01:05:17.587938 140576550393728 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0

In [0]:
train_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/train'
validation_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/validation'

train_datagen = ImageDataGenerator(
  vertical_flip = True,
  horizontal_flip = True
)
validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size = (128,128),
  batch_size = 16,
  class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
  validation_dir,
  target_size = (128,128),
  batch_size = 16,
  class_mode = 'binary'
)

model.compile(loss='binary_crossentropy',
              optimizer= 'adadelta',
              metrics=['accuracy'])

best_model_VA = keras.callbacks.ModelCheckpoint('BM_VA_'+sys.argv[1],monitor='val_acc',
                                mode = 'max', verbose=1, save_best_only=True)
best_model_VL = keras.callbacks.ModelCheckpoint('BM_VL_'+sys.argv[1],monitor='val_loss',
                                mode = 'min', verbose=1, save_best_only=True)

Found 882 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


W0717 01:05:18.073258 140576550393728 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
history = model.fit_generator(train_generator, steps_per_epoch=55, epochs=100, verbose=1, callbacks=None,
                              validation_data=validation_generator, validation_steps=7, validation_freq=1)

Epoch 1/100
55/55 [==============================] - 223s 4s/step - loss: 16.8849 - acc: 0.5092 - val_loss: 3.2680 - val_acc: 0.5189
Epoch 2/100
55/55 [==============================] - 35s 639ms/step - loss: 13.9945 - acc: 0.5254 - val_loss: 2.5378 - val_acc: 0.5943
Epoch 3/100
55/55 [==============================] - 35s 639ms/step - loss: 13.7132 - acc: 0.5219 - val_loss: 3.4763 - val_acc: 0.5283
Epoch 4/100
55/55 [==============================] - 35s 636ms/step - loss: 10.9560 - acc: 0.5277 - val_loss: 2.8235 - val_acc: 0.5189
Epoch 5/100
55/55 [==============================] - 35s 635ms/step - loss: 10.5117 - acc: 0.4954 - val_loss: 2.9667 - val_acc: 0.5472
Epoch 6/100
55/55 [==============================] - 35s 632ms/step - loss: 7.8181 - acc: 0.5591 - val_loss: 2.0580 - val_acc: 0.5755
Epoch 7/100
55/55 [==============================] - 35s 630ms/step - loss: 6.2492 - acc: 0.5469 - val_loss: 1.8424 - val_acc: 0.5943
Epoch 8/100
55/55 [==============================] - 34s 62

# VGG16 + Dense Layer

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import os
import sys
from tensorflow.keras import layers
from tensorflow.keras import models

In [0]:
# Train Model with VGG16


local_file = '/content/skincancer/models/model_2_vgg.h5'

pre_trained_model = keras.applications.vgg16.VGG16(
    input_shape=(128,128,3),
    include_top=False,
    weights = 'imagenet'
)

for layer in pre_trained_model.layers:
  layer.trainable=False

 
  
pre_trained_model.summary()

last_layer = pre_trained_model.output

top_model = layers.Flatten()(last_layer)
top_model = layers.Dense(128,activation='relu')(top_model)
top_model = layers.Dropout(.5)(top_model)
top_model = layers.Dense(1,activation='sigmoid')(top_model)

model = models.Model(pre_trained_model.input,top_model)

model.load_weights(local_file) 

model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.SGD(lr=1e-4,momentum=0.9),
    metrics = ['accuracy']
)

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [0]:
train_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/train'
validation_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/validation'

train_datagen = ImageDataGenerator(
  vertical_flip = True,
  horizontal_flip = True
)
validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size = (128,128),
  batch_size = 16,
  class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
  validation_dir,
  target_size = (128,128),
  batch_size = 16,
  class_mode = 'binary'
)

best_model_VA = keras.callbacks.ModelCheckpoint('BM_VA_'+sys.argv[1],monitor='val_acc',
                                mode = 'max', verbose=1, save_best_only=True)
best_model_VL = keras.callbacks.ModelCheckpoint('BM_VL_'+sys.argv[1],monitor='val_loss',
                                mode = 'min', verbose=1, save_best_only=True)

Found 882 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [0]:
history = model.fit_generator(train_generator, steps_per_epoch=55, epochs=100, verbose=1, callbacks=None,
                              validation_data=validation_generator, validation_steps=7, validation_freq=1)

Epoch 1/100
55/55 [==============================] - 37s 675ms/step - loss: 0.6452 - acc: 0.7875 - val_loss: 0.1864 - val_acc: 0.9340
Epoch 2/100
55/55 [==============================] - 36s 649ms/step - loss: 0.5751 - acc: 0.7933 - val_loss: 0.2493 - val_acc: 0.8774
Epoch 3/100
55/55 [==============================] - 36s 650ms/step - loss: 0.4349 - acc: 0.8014 - val_loss: 0.2583 - val_acc: 0.8962
Epoch 4/100
55/55 [==============================] - 35s 635ms/step - loss: 0.4377 - acc: 0.8037 - val_loss: 0.2613 - val_acc: 0.8962
Epoch 5/100
55/55 [==============================] - 36s 653ms/step - loss: 0.4446 - acc: 0.8095 - val_loss: 0.2439 - val_acc: 0.9245
Epoch 6/100
55/55 [==============================] - 36s 650ms/step - loss: 0.3867 - acc: 0.8216 - val_loss: 0.2568 - val_acc: 0.8585
Epoch 7/100
55/55 [==============================] - 35s 639ms/step - loss: 0.3913 - acc: 0.8310 - val_loss: 0.2787 - val_acc: 0.8491
Epoch 8/100
55/55 [==============================] - 35s 636ms

# CNN + Data Augmentation

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import sys

In [0]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),     # Dropout layer for Overfittion
    keras.layers.Dense(1, activation = 'sigmoid')    
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12544)            

In [0]:
train_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/train'
validation_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/validation'

train_datagen = ImageDataGenerator(
  rotation_range = 180,
  vertical_flip = True,
  horizontal_flip = True
)
validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size = (128,128),
  batch_size = 16,
  class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
  validation_dir,
  target_size = (128,128),
  batch_size = 16,
  class_mode = 'binary'
)

model.compile(loss='binary_crossentropy',
              optimizer= 'adadelta',
              metrics=['accuracy'])

best_model_VA = keras.callbacks.ModelCheckpoint('BM_VA_'+sys.argv[1],monitor='val_acc',
                                mode = 'max', verbose=1, save_best_only=True)
best_model_VL = keras.callbacks.ModelCheckpoint('BM_VL_'+sys.argv[1],monitor='val_loss',
                                mode = 'min', verbose=1, save_best_only=True)

Found 882 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [0]:
history = model.fit_generator(train_generator, steps_per_epoch=55, epochs=100, verbose=1, callbacks=None,
                              validation_data=validation_generator, validation_steps=7, validation_freq=1)

Epoch 1/100
55/55 [==============================] - 38s 684ms/step - loss: 11.6532 - acc: 0.4642 - val_loss: 3.6024 - val_acc: 0.4057
Epoch 2/100
55/55 [==============================] - 38s 686ms/step - loss: 9.4176 - acc: 0.4804 - val_loss: 3.4542 - val_acc: 0.4245
Epoch 3/100
55/55 [==============================] - 37s 670ms/step - loss: 8.9048 - acc: 0.4804 - val_loss: 2.9181 - val_acc: 0.3774
Epoch 4/100
55/55 [==============================] - 37s 669ms/step - loss: 7.6358 - acc: 0.4919 - val_loss: 2.8265 - val_acc: 0.4623
Epoch 5/100
55/55 [==============================] - 37s 679ms/step - loss: 5.7207 - acc: 0.4977 - val_loss: 2.7224 - val_acc: 0.4340
Epoch 6/100
55/55 [==============================] - 37s 670ms/step - loss: 5.7750 - acc: 0.4977 - val_loss: 2.0639 - val_acc: 0.3868
Epoch 7/100
55/55 [==============================] - 36s 659ms/step - loss: 4.5996 - acc: 0.5104 - val_loss: 2.0179 - val_acc: 0.4245
Epoch 8/100
55/55 [==============================] - 36s 661m

# CNN + VGG16 + Data Augmentation

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import sys
import os
from tensorflow.keras import layers
from tensorflow.keras import models

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Train Model with VGG16

local_file = '/content/skincancer/models/model_2_vgg.h5'

pre_trained_model = keras.applications.vgg16.VGG16(
    input_shape=(128,128,3),
    include_top=False,
    weights = 'imagenet'
)

for layer in pre_trained_model.layers:
  layer.trainable=False

 
  
pre_trained_model.summary()

last_layer = pre_trained_model.output

top_model = layers.Flatten()(last_layer)
top_model = layers.Dense(128,activation='relu')(top_model)
top_model = layers.Dropout(.5)(top_model)
top_model = layers.Dense(1,activation='sigmoid')(top_model)

model = models.Model(pre_trained_model.input,top_model)

model.load_weights(local_file) 

model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.SGD(lr=1e-4,momentum=0.9),
    metrics = ['accuracy']
)

W0727 17:09:04.941860 139690886772608 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


58892288/58889256 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
______________________________________________________________

OSError: ignored

In [0]:
train_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/train'
validation_dir = '/content/drive/My Drive/Datasets/Skin_Data/Skin_Detection/validation'

train_datagen = ImageDataGenerator(
  rescale = 1./255,
  rotation_range = 180,
  vertical_flip = True,
  horizontal_flip = True,
)
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size = (128,128),
  batch_size = 16,
  class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
  validation_dir,
  target_size = (128,128),
  batch_size = 16,
  class_mode = 'binary'
)

best_model_VA = keras.callbacks.ModelCheckpoint('BM_VA_'+sys.argv[1],monitor='val_acc',
                                mode = 'max', verbose=1, save_best_only=True)
best_model_VL = keras.callbacks.ModelCheckpoint('BM_VL_'+sys.argv[1],monitor='val_loss',
                                mode = 'min', verbose=1, save_best_only=True)

Found 882 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [0]:
history = model.fit_generator(train_generator, steps_per_epoch=55, epochs=100, verbose=1, callbacks=None,
                              validation_data=validation_generator, validation_steps=7, validation_freq=1)

Epoch 1/100
55/55 [==============================] - 409s 7s/step - loss: 0.6630 - accuracy: 0.6097 - val_loss: 0.6499 - val_accuracy: 0.6038
Epoch 2/100
55/55 [==============================] - 171s 3s/step - loss: 0.6496 - accuracy: 0.6212 - val_loss: 0.6574 - val_accuracy: 0.5755
Epoch 3/100
55/55 [==============================] - 167s 3s/step - loss: 0.6544 - accuracy: 0.6005 - val_loss: 0.6580 - val_accuracy: 0.5755
Epoch 4/100
55/55 [==============================] - 166s 3s/step - loss: 0.6408 - accuracy: 0.6374 - val_loss: 0.6473 - val_accuracy: 0.5755
Epoch 5/100
55/55 [==============================] - 164s 3s/step - loss: 0.6264 - accuracy: 0.6467 - val_loss: 0.6635 - val_accuracy: 0.6226
Epoch 6/100
55/55 [==============================] - 164s 3s/step - loss: 0.6372 - accuracy: 0.6467 - val_loss: 0.6421 - val_accuracy: 0.6887
Epoch 7/100
55/55 [==============================] - 162s 3s/step - loss: 0.6379 - accuracy: 0.6351 - val_loss: 0.6486 - val_accuracy: 0.6226
Epoch 